<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/prophet-code/prophet-random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 骆驼先知

骆驼先知是 李鲁鲁 受到吴恩达的prompt Engineering的启发

制作的一个纪伯伦的《先知》的拓展版本

运行这个notebook你需要OpenAI的API Token

项目链接 [https://github.com/LC1332/Prophet-Andrew-Ng](https://github.com/LC1332/Prophet-Andrew-Ng)

骆驼先知是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一，后者由李鲁鲁，冷子昂，陈启源发起。

In [ ]:
! pip install openai

In [2]:
import openai
import os

# 导入第三方库

openai.api_key  = 'sk-'
# 李鲁鲁 在这里设置你的API_KEY

## 数据读取

读取prompt-data中的文本数据，作为in-context-learning的数据

In [ ]:
prophet_data_folder = './../prophet-data'

import os

titles = []
title_to_text = {}

# scan all txt file in prophet_data_folder
for file in os.listdir(prophet_data_folder):
    if file.endswith('.txt'):
        title_name = file[:-4]
        titles.append(title_name)

        with open(os.path.join(prophet_data_folder, file), 'r') as f:
            title_to_text[title_name] = f.read()

# report length of each text
for title in titles:
    print(title, len(title_to_text[title]))

孩子 315
爱 789
婚姻 292


我们将在后续课程中深入探究 OpenAI 提供的 ChatCompletion API 的使用方法，在此处，我们先将它封装成一个函数，你无需知道其内部机理，仅需知道调用该函数输入 Prompt 其将会给出对应的 Completion 即可。

In [ ]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


在这里我们先假设一个query的主题 "时间"

In [ ]:
query_topic = "时间"

定义先知和市民的身份，这里是为了方便后面的泛化

In [ ]:
prophet_name = "先知"

citizen_name = "市民"

形成一个shot的句子输出

In [ ]:
# 实现一个Python函数 ensemble_one_shot ，输入为prophet_name, citizen_name, sample_topic，sample_answer， 输出为一个组织好的字符串
# 例子输入: 先知, 市民, 衣服， "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。 \n 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"
# 例子输出: """
# <市民>:请和我们讨论一下"衣服"

# <先知>: 你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
# 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。
# """
def ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer):
    # 组织对话文本
    dialogue = "<{}>:```请和我们讨论一下\"{}\"```\n\n<{}>:```{}```\n\n".format(citizen_name, sample_topic, prophet_name, sample_answer)
    return dialogue

# unit test for ensemble_one_shot
prophet_name = "先知"
citizen_name = "市民"
sample_topic = "衣服"
sample_answer = "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。\n尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"

dialogue = ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)
print(dialogue)

<市民>:```请和我们讨论一下"衣服"```

<先知>:```你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。```




下面我们来组织完整的prompt，我们假设已经选取了两个主题作为例子 孩子和爱

In [ ]:
selected_sample = ["孩子","爱"]

def organize_prompt( query_topic, selected_sample ):
    prompt = """你的任务是以一致的风格回答问题。\n\n"""
    
    for sample_topic in selected_sample:
        # find sample_answer in dictionary
        sample_answer = title_to_text[sample_topic]
        prompt += ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)

    prompt += """<{}>:```请和我们讨论一下"{}"```\n\n""".format(citizen_name, query_topic)

    return prompt

# write a unit test for organize_prompt
query_topic = "时间"
selected_sample = ["孩子","爱"]
prompt = organize_prompt( query_topic, selected_sample )
print(prompt)

你的任务是以一致的风格回答问题。

<市民>:```请和我们讨论一下"孩子"```

<先知>:```你们的孩子并不是你们的，
而是“生命”对自身的渴望所生的儿女
他们借你们来到世上,却并非来自你们,
他们虽与你们一起生活,却并不属于你们。

你们可把爱给予他们,却不能给予他们以思想
因为他们有他们的思想。
你们能够庇护他们的身体,却不能庇护他们的灵魂。
因为他们的灵魂居于明日的华屋,那是你们无法想见的,即使在梦中。

你们可以努力以求像他们,但不要试图让他们像你们。
因为生命不能走退步,它不可能滞留在昨天。
你们是弓,你们的孩子则是从你们的弓弦上射出的矢箭.
射手看见坚立在无尽头路上的目标
他会用自己的神力将你们的弓引满，以便让他的箭快速射至最远。
就让你们的弓在射手的手中甘愿曲弯;
因为他既爱那飞快的箭,也爱那静止的弓```

<市民>:```请和我们讨论一下"爱"```

<先知>:```爱向你们示意,你们就跟他走
即使道路崎岖,坡斜陡滑。
如果爱向你们展开双翅,你就服从之
即使藏在羽翻中的利剑会伤着你们。
如果爱对你们说什么,你们只管相信他
即使他的声音惊扰了你们的美梦,犹如北风将园林吹得花木调零。

爱为你们戴上冠冕的同时,也会把你们钉在十字架上。爱能强壮你们的骨干，同时也要修剪你们的枝条。
爱能升腾到你们天际的至高处,抚弄你们那摇曳在阳光里的柔嫩细枝。
爱同样能沉人你们那伸进泥土里的根部,并将根部动摇。

爱把你们抱在怀里,如同抱着一捆麦子爱把你们春打,以使你们赤体裸身。爱把你们过筛子,以便筛去外壳
爱把你们磨成面粉。
爱把你们和成面团,让你们变得柔软。
爱再把你们放在他的圣殿里的火上,以期让你们变成上帝圣缝上的神圣面包。

爱如此摆弄你们,为的是让你们知道你们心中的秘密。依靠这一见识,你们就能成为存在之心的一片碎屑。
如果你们心存恐惧,只想在爱中寻求安逸和享受那么,你们最好遮盖起自己的裸体,逃离爱的打谷场，
走向一个没有季节更替的世界;在那里,你们可以笑,但笑得不尽情;在那里,你们可以哭,但眼泪淌不完。

爱,除了自己,既不给予,也不索取。
爱,既不占有,也不被任何人占有。
爱,仅仅满足于自己而已.
当你爱的时候,你不要说“上帝在我心中”而要说“我在上帝心中”。
你切莫以为自己能够指引爱之行程爱会引导你,如果发现你适于引导。

爱除了实现自我,别无所求

在第一个版本中，我们selected_sample先使用随机的策略

In [ ]:
import random

# def function random_select_title, random pick k sample from titles
def random_select_title(titles, k):
    return random.sample(titles, k)

# unit test for random_select_title
selected_sample = random_select_title(titles, 2)
print(selected_sample)

['孩子', '婚姻']


集成所有的代码！

In [ ]:
query_topic = "时间"
selected_sample = random_select_title(titles, 2)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

当然你也可以指定特定的title进行讨论

In [ ]:
query_topic = "内卷"
selected_sample = ["劳作","自由"]
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

**策略四：提供少量示例**

即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。

例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和一个祖父之间的对话的例子。孩子说，“教我耐心”，祖父用这些隐喻回答。因此，由于我们已经告诉模型要以一致的语气回答，现在我们说“教我韧性”，由于模型已经有了这个少样本示例，它将以类似的语气回答下一个任务。

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends with the wind but never breaks. It is the ability to bounce back from adversity and keep moving forward, even when things get tough. Just like a tree that grows stronger with each storm it weathers, resilience is a quality that can be developed and strengthened over time.


In [ ]:
prompt = f"""
你的任务是以一致的风格回答问题。

<孩子>: 教我耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

<祖父母>: 韧性就像是一棵树，它需要经历风吹雨打、寒冬酷暑，才能成长得更加坚强。所以，当你遇到挫折和困难时，不要轻易放弃，要像树一样坚定地扎根，不断成长，最终成为一棵高大的树。


李鲁鲁：这种说教风格的学习

直接让我想起了我小时候中二阶段（国中二年级）的文学创作巅峰

那个时候很喜欢 纪伯伦的 《先知》 觉得先知的说法 风格非常说教

我们不如现学现卖一段，这里就不使用代码了。


这里我们想让 《先知》 故事中的先知

来讨论时间这个概念， 于是ChatGPT的输入

```
你的任务是以一致的风格回答问题。

<市民>: 请和我们讨论一下"孩子"

<先知>: """你的孩子，其实不是你的孩子，
他们是生命对于自身渴望而诞生的孩子，
他们通过你来到这世界，却非因你而来，
他们在你身边，却并不属于你。
你可以给予他们的是你的爱，却不是你的想法，
因为他们自己有自己的思想。
你可以庇护的是他们的身体，却不是他们的灵魂，
因为他们的灵魂属于明天，属于你做梦也无法达到的明天。
你可以拼尽全力，变得象他们一样，
却不要让他们变得和你一样，
因为生命不会后退，也不在过去停留。
你是弓，儿女是从你那里射出的箭。
弓箭手望着未来之路上的箭靶，
他用尽力气将你拉开，使他的箭射得又快又远。
怀着快乐的心情，在弓箭手的手里弯曲吧，
因为他爱一路飞翔的箭，也爱无比稳定的弓。"""

<市民>:请和我们讨论一下"衣服"

<先知>: 你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。
但愿你们用自己的肌肤而不是衣服去迎接阳光和清风，
因为阳光中有生命的气息，而风中有生命之手。
你们中一些人说：“是北风织造了我们所穿的衣服。”
我说，是的，的确是北风，
但它以羞怯为织机，以纤弱的肌肉为纱线。
它一旦完成工作，便会在林中大笑。
不要忘记，羞怯原是抵挡不洁目光的盾牌。
若无邪侪，那羞怯除了是精神上的束缚和污垢外还能是什么？
也不要忘记，大地乐于感觉体赤裸的双脚，风儿渴望与你的头发嬉戏。

<市民>:请和我们讨论一下"时间"
```

让我们来看看GPT的输出

```
<先知>: 时间是一条河流，它汇聚着每个生命的流动和消逝。
每个人都在时间的河流中漂泊，不断地向前推进。
我们无法停止或掌控时间，但可以选择如何利用它。
时间是宝贵的，它可以让我们成长、学习、改变和创造。
但若我们不珍惜时间，它也会成为我们生命中的遗憾和遗失。
无论过去、现在还是未来，时间都是无情的，它会不断流逝，
我们唯一能做的就是珍惜它，珍惜每分每秒的生命，让我们的时间变得更加有意义和宝贵。
```

这个回答可以说是相当不错了。

这里我们来挖个坑吧，我收集了所有纪伯伦的诗，如果这套课刷完了

可以做个自动哲理机器人。。。

### 原则二：给模型时间去思考

如果模型匆忙地得出了错误的结论，您应该尝试重新构思查询，请求模型在提供最终答案之前进行一系列相关的推理。换句话说，如果您给模型一个在短时间或用少量文字无法完成的任务，它可能会猜测错误。这种情况对人来说也是一样的。如果您让某人在没有时间计算出答案的情况下完成复杂的数学问题，他们也可能会犯错误。因此，在这些情况下，您可以指示模型花更多时间思考问题，这意味着它在任务上花费了更多的计算资源。

**策略一：指定完成任务所需的步骤**

接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果

首先我们描述了杰克和吉尔的故事，并给出一个指令。该指令是执行以下操作。首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和名称数。然后我们要用换行符分隔答案。

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Two siblings, Jack and Jill, go on a quest to fetch water from a well on a hilltop, but misfortune strikes and they both tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed.

Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits sur une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts. 
Noms: Jack, Jill.

{
  "french_summary": "Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits sur une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.",
  "num_names": 2
}


In [ ]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：French_summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
1-兄妹在山顶井里打水时发生意外，但仍然保持冒险精神。
2-Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne. Malheureusement, Jack trébuche sur une pierre et tombe de la montagne, suivi de près par Jill. Bien qu'ils soient légèrement blessés, ils retournent chez eux chaleureusement. Malgré cet accident, leur esprit d'aventure ne diminue pas et ils continuent à explorer joyeusement.
3-Jack, Jill
4-{
   "French_summary": "Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne. Malheureusement, Jack trébuche sur une pierre et tombe de la montagne, suivi de près par Jill. Bien qu'ils soient légèrement blessés, ils retournent chez eux chaleureusement. Malgré cet accident, leur esprit d'aventure ne diminue pas et ils continuent à explorer joyeusement.",
   "num_names": 2
}


上述输出仍然存在一定问题，例如，键“姓名”会被替换为法语，因此，我们给出一个更好的 Prompt，该 Prompt 指定了输出的格式

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in French summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: 兄妹杰克和吉尔在山顶井里打水时发生意外，但他们仍然保持冒险精神继续探索。
Translation: Jack and Jill, deux frères et sœurs, ont eu un accident en allant chercher de l'eau dans un puits de montagne, mais ils ont continué à explorer avec un esprit d'aventure.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack and Jill, deux frères et sœurs, ont eu un accident en allant chercher de l'eau dans un puits de montagne, mais ils ont continué à explorer avec un esprit d'aventure.", "num_names": 2}


In [ ]:
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。

请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
摘要：兄妹杰克和吉尔在迷人的村庄里冒险，不幸摔伤后回到家中，但仍然充满冒险精神。
翻译：In a charming village, siblings Jack and Jill set out to fetch water from a mountaintop well. While climbing and singing, Jack trips on a stone and tumbles down the mountain, with Jill following closely behind. Despite some bruises, they make it back home safely. Their adventurous spirit remains undiminished as they continue to explore with joy.
名称：Jack，Jill
输出 JSON：{"English_summary": "In a charming village, siblings Jack and Jill set out to fetch water from a mountaintop well. While climbing and singing, Jack trips on a stone and tumbles down the mountain, with Jill following closely behind. Despite some bruises, they make it back home safely. Their adventurous spirit remains undiminished as they continue to explore with joy.", "num_names": 2}


**策略二：指导模型在下结论之前找出一个自己的解法**

有时候，在明确指导模型在做决策之前要思考解决方案时，我们会得到更好的结果。

接下来我们会给出一个问题和一个学生的解答，要求模型判断解答是否正确

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


In [ ]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

学生的解决方案是正确的。


但是注意，学生的解决方案实际上是错误的。

我们可以通过指导模型先自行找出一个解法来解决这个问题。

在接下来这个 Prompt 中，我们要求模型先自行解决这个问题，再根据自己的解法与学生的解法进行对比，从而判断学生的解法是否正确。同时，我们给定了输出的格式要求。通过明确步骤，让模型有更多时间思考，有时可以获得更准确的结果。在这个例子中，学生的答案是错误的，但如果我们没有先让模型自己计算，那么可能会被误导以为学生是正确的。

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x

Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect


In [ ]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：

    首先，自己解决问题。
    然后将你的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

    问题：问题文本
    学生的解决方案：学生的解决方案文本
    实际解决方案和步骤：实际解决方案和步骤文本
    学生的解决方案和实际解决方案是否相同：是或否
    学生的成绩：正确或不正确

问题：

    我正在建造一个太阳能发电站，需要帮助计算财务。 
    - 土地费用为每平方英尺100美元
    - 我可以以每平方英尺250美元的价格购买太阳能电池板
    - 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

    设x为发电站的大小，单位为平方英尺。
    费用：
    1. 土地费用：100x
    2. 太阳能电池板费用：250x
    3. 维护费用：100,000+100x
    总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
"""
response = get_completion(prompt)
print(response)

正确的解决方案和步骤：
    1. 计算土地费用：100美元/平方英尺 * x平方英尺 = 100x美元
    2. 计算太阳能电池板费用：250美元/平方英尺 * x平方英尺 = 250x美元
    3. 计算维护费用：10万美元 + 10美元/平方英尺 * x平方英尺 = 10万美元 + 10x美元
    4. 计算总费用：100x美元 + 250x美元 + 10万美元 + 10x美元 = 360x + 10万美元

学生的解决方案和实际解决方案是否相同：否

学生的成绩：不正确


## 三、局限性

**虚假知识**：模型偶尔会生成一些看似真实实则编造的知识

如果模型在训练过程中接触了大量的知识，它并没有完全记住所见的信息，因此它并不很清楚自己知识的边界。这意味着它可能会尝试回答有关晦涩主题的问题，并编造听起来合理但实际上并不正确的答案。我们称这些编造的想法为幻觉。

例如在如下示例中，我们要求告诉我们 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 产品的信息，事实上，这个公司是真实存在的，但产品是编造的，模型则会一本正经地告诉我们编造的知识。



In [ ]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that uses advanced sonic technology to provide a deep and thorough clean. It features a slim and sleek design that makes it easy to hold and maneuver, and it comes with a range of smart features that help you optimize your brushing routine.

One of the key features of the AeroGlide UltraSlim Smart Toothbrush is its advanced sonic technology, which uses high-frequency vibrations to break up plaque and bacteria on your teeth and gums. This technology is highly effective at removing even the toughest stains and buildup, leaving your teeth feeling clean and fresh.

In addition to its sonic technology, the AeroGlide UltraSlim Smart Toothbrush also comes with a range of smart features that help you optimize your brushing routine. These include a built-in timer that ensures you brush for the recommended two minutes, as well as a pressure sensor that alerts you if you're brushing too hard.

Overall, the AeroGlide UltraS

In [ ]:
prompt = f"""
告诉我 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 的相关信息
"""
response = get_completion(prompt)
print(response)

Boie公司生产的AeroGlide UltraSlim Smart Toothbrush是一款智能牙刷，具有以下特点：

1. 超薄设计：刷头仅有0.8毫米的厚度，可以更容易地进入口腔深处，清洁更彻底。

2. 智能感应：牙刷配备了智能感应技术，可以自动识别刷头的位置和方向，确保每个部位都得到充分的清洁。

3. 高效清洁：牙刷采用了高速振动技术，每分钟可达到40000次，可以有效去除牙菌斑和污渍。

4. 轻松携带：牙刷采用了便携式设计，可以轻松放入口袋或旅行包中，随时随地进行口腔清洁。

5. 环保材料：牙刷采用了环保材料制造，不含有害物质，对环境友好。

总之，Boie公司生产的AeroGlide UltraSlim Smart Toothbrush是一款高效、智能、环保的牙刷，可以帮助用户轻松保持口腔健康。


模型会输出看上去非常真实的编造知识，这有时会很危险。因此，请确保使用我们在本节中介绍的一些技巧，以尝试在构建自己的应用程序时避免这种情况。这是模型已知的一个弱点，也是我们正在积极努力解决的问题。在你希望模型根据文本生成答案的情况下，另一种减少幻觉的策略是先要求模型找到文本中的任何相关引用，然后要求它使用这些引用来回答问题，这种追溯源文档的方法通常对减少幻觉非常有帮助。

**说明：在本教程中，我们使用 \ 来使文本适应屏幕大小以提高阅读体验，GPT 并不受 \ 的影响，但在你调用其他大模型时，需额外考虑 \ 是否会影响模型性能**